In [ ]:
# 1. Go to /content
%cd /content

# 2. Remove old yolov7 (optional, if it's half-installed)
!rm -rf yolov7

# 3. Clone YOLOv7
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

# 4. Install a compatible numpy version
!pip install numpy==1.26.4

# 5. Install requirements (ignore numpy line in requirements.txt)
!pip install -r requirements.txt --no-deps

# 6. Install latest supported torch + torchvision for CUDA 11.8
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu118


In [ ]:
# 7. Download original yolov7.pt weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt


In [ ]:
!python detect.py \
    --weights yolov7.pt \
    --source /content/Bike_Polo.mp4 \
    --img-size 640 \
    --conf-thres 0.10 \
    --iou-thres 0.45 \
    --save-txt \
    --save-conf \
    --project runs/detect \
    --name exp \
    --exist-ok



In [ ]:

# 1. Import necessary libraries
import os
import pandas as pd

# 2. Set Detections Folder Path
detections_folder = '/content/yolov7/runs/detect/exp/labels/'  # corrected path

# 3. Load Annotations (ground-truth labels)
annotation_df = pd.read_csv('/content/Bike_Polo.csv')
frame_labels = annotation_df['label'].tolist()

# 4. Check detection files in the directory
files = os.listdir(detections_folder)

txt_files = [f for f in files if f.endswith('.txt')]

# 5. Print detection files information
print(f"✅ Total .txt detection files: {len(txt_files)}")
print("First 10 detection files:", txt_files[:10])

# 6. Print ground-truth labels information
print(f"✅ Total ground-truth labels: {len(frame_labels)}")
print(" ground-truth labels:", frame_labels[:3000])




In [ ]:
import os

# How many detection txt files generated?
print("Number of detection label files:", len(os.listdir(detections_folder)))

# Show a few sample detection contents
for file_name in os.listdir(detections_folder)[:5]:  # First 5 files
    print(f"\nFile: {file_name}")
    with open(os.path.join(detections_folder, file_name), 'r') as file:
        print(file.read())


In [ ]:
import os

# 1. Define object weights for each class (example weights)
# Replace this with the actual class weights for your use case
object_weight = {
    0: 1.0,  # Class 0 weight
    1: 1.5,  # Class 1 weight
    2: 2.0,  # Class 2 weight
    # Add more classes here
}

# 2. Initialize frame scores list
frame_scores = []

# 3. Loop through each frame index and calculate frame score
for frame_idx in range(len(frame_labels)):
    # Adjust the filename format to match your detected files
    frame_file = os.path.join(detections_folder, f'Bike_Polo_{frame_idx + 1}.txt')  # No leading zeros

    frame_score = 0.0

    # Check if the detection file exists
    if os.path.exists(frame_file):
        with open(frame_file, 'r') as f:
            lines = f.readlines()

            # Check if the file is empty
            if len(lines) == 0:
                print(f"⚠️ Frame {frame_idx + 1}: Detection file exists but is EMPTY.")

            for line in lines:
                parts = line.strip().split()

                if len(parts) >= 6:
                    cls = int(parts[0])  # Class ID
                    conf = float(parts[5])  # Confidence score

                    # Debugging: Print detected class and confidence
                    print(f"Frame {frame_idx + 1}: Detected class {cls}, Confidence {conf}")

                    weight = object_weight.get(cls, None)  # Get class weight

                    if weight is None:
                        # Handle unknown class
                        print(f"❗ Frame {frame_idx + 1}: Unknown class {cls} detected. Skipping.")
                        continue  # Skip this detection

                    # Debugging: Print the weight for each detected class
                    print(f"Frame {frame_idx + 1}: Class {cls}, Weight: {weight}")

                    # Add weighted confidence to the frame score
                    frame_score += weight * conf
                else:
                    print(f"⚠️ Frame {frame_idx + 1}: Invalid detection format in line: {line}")

    else:
        print(f"❗ Frame {frame_idx + 1}: Detection file {frame_file} not found.")

    # Add the calculated frame score to the list
    frame_scores.append(frame_score)

# 4. Print a sample of calculated frame scores
print("\n✅ Frame scores calculated.")
print("First 1000 frame scores:", frame_scores[:3000])


In [ ]:
import numpy as np
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,
    classification_report, roc_auc_score, precision_recall_curve
)
import matplotlib.pyplot as plt
import seaborn as sns

# Convert frame_scores to numpy array
frame_scores = np.array(frame_scores)

# Ground truth labels
ground_truth = np.array(frame_labels)



# Threshold = mean + 0.5 * std
threshold = np.mean(frame_scores) + 0.5* np.std(frame_scores)
print(f"Threshold value: {threshold:.4f}")

# Predicted labels
predicted_labels = (frame_scores > threshold).astype(int)

# Convert predicted labels to list
predicted_list = predicted_labels.tolist()

# Print predicted labels in chunks of 20
print("\nPredicted Labels List (formatted in rows of 20):")
for i in range(0, len(predicted_list), 20):
    print(predicted_list[i:i+20])

In [ ]:
import cv2
import os

# Load your original video
input_video_path = "/content/Bike_Polo.mp4"  # Path to your original video
cap = cv2.VideoCapture(input_video_path)

# Get the directory of the original video file
video_directory = os.path.dirname(input_video_path)

# Video properties: to maintain the same frame rate and resolution
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video setup, saving it in the same directory as the original video
output_video_path = os.path.join(video_directory, "summarized_video.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_index = 0
good_frame_index = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Check if frame_index is within the bounds of predicted_labels
    if frame_index < len(predicted_labels):
        # Check if the current frame is good (based on predicted label)
        if predicted_labels[frame_index] == 1:
            out.write(frame)  # Write the frame to the new video
            good_frame_index += 1
    else:
        # If frame_index is out of bounds, break the loop
        break
    frame_index += 1


# Release the video capture and writer
cap.release()
out.release()

print(f"Summarized video saved as {output_video_path} with {good_frame_index} good frames.")

In [ ]:
# Define the input video path
input_video_path = '/content/summarized_video.mp4'  # Update if the path is different

# Run YOLOv7 detection on the summarized video
!python detect.py \
    --weights yolov7.pt \
    --conf 0.25 \
    --source $input_video_path \
    --save-txt \
    --save-conf \
    --project runs/detect \
    --name final_video \
    --exist-ok

# Move the detected video to a convenient location and rename
import shutil
shutil.move('runs/detect/final_video/summarized_video.mp4', '/content/final_video.mp4')

# Display the final video in the notebook
from IPython.display import Video
Video('/content/final_video.mp4', embed=True)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Compute evaluation metrics
cm = confusion_matrix(ground_truth, predicted_labels)
acc = accuracy_score(ground_truth, predicted_labels)
prec = precision_score(ground_truth, predicted_labels)
rec = recall_score(ground_truth, predicted_labels)
f1 = f1_score(ground_truth, predicted_labels)

# Display metrics
print("\nConfusion Matrix:")
print(cm)
print(f"\nAccuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

# Classification Report (optional for better insight)
print("\nClassification Report:")
print(classification_report(ground_truth, predicted_labels))

# Confusion Matrix with custom labels
tn, fp, fn, tp = cm.ravel()  # Unpack confusion matrix into TP, FP, FN, TN

# 1. Plot Confusion Matrix with custom labels
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Bad Frame (0)', 'Good Frame (1)'],
            yticklabels=['Bad Frame (0)', 'Good Frame (1)'], linewidths=0.5)

# Customizing the plot labels
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.title(f"Confusion Matrix\n(TP={tp}, FP={fp}, FN={fn}, TN={tn})", fontsize=14)

# Show the plot
plt.show()


